In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
font = {'family' : 'normal',
        'size'   : 22}	
mpl.rc('font', **font)

In [30]:
df = pd.read_csv("./bases/suicide/master.csv") 
filtro = df.year > 1999
filtro2 = df.year < 2016
df = df[filtro & filtro2]
df = df.groupby(['country','year','age'], as_index=False)['suicides_no','population'].sum()
df = df.set_index(['country','year','age'])
df = df.reindex( pd.MultiIndex.from_product([df.index.levels[0],df.index.levels[1],df.index.levels[2]], names=['country', 'year','age']),fill_value=np.nan).reset_index()
func = lambda x:x.suicides_no.isna()
series = df[func].reset_index().groupby(['country'])['year'].count()
lista = list(series[series > 30].index)
filtro3 = df.country.isin(lista)
df = df[~filtro3]
def limpa_idade(age):
    return age[:-6]
df['age'] = df['age'].apply(limpa_idade)
df

,country,year,age,suicides_no,population
0,Albania,2000,15-24,11.0,503900.0
1,Albania,2000,25-34,20.0,477800.0
2,Albania,2000,35-54,9.0,706900.0
3,Albania,2000,5-14,1.0,699600.0
4,Albania,2000,55-74,10.0,345400.0
5,Albania,2000,75+,3.0,62700.0
6,Albania,2001,15-24,19.0,527398.0
7,Albania,2001,25-34,26.0,429255.0
8,Albania,2001,35-54,46.0,749017.0
9,Albania,2001,5-14,8.0,628912.0


In [29]:
def preenche_nan(df):
    rdfs= pd.melt(df.pivot_table('suicides_no', ['country'], 'year')
    .interpolate(method='linear',limit_direction='both', axis=1).reset_index(),
    id_vars=["country"], var_name="year", value_name="suicides")
    rdfp = pd.melt(df.pivot_table('population', ['country'], 'year')
    .interpolate(method='linear',limit_direction='both', axis=1).reset_index(),
    id_vars=["country"], var_name="year", value_name="population")
    result = rdfs.merge(rdfp).sort_values(['country','year']).reset_index(drop=True)
    result['suicide100'] = (result['suicides'] * 100000 ) / result['population']
    return result
df1 = preenche_nan(df[df.age == '5-14'])
df2 = preenche_nan(df[df.age == '15-24'])
df3 = preenche_nan(df[df.age == '25-34'])
df4 = preenche_nan(df[df.age == '35-54'])
df5 = preenche_nan(df[df.age == '55-74'])
df6 = preenche_nan(df[df.age == '75+'])

,country,year,suicides,population,suicide100
0,Albania,2000,1.00,699600.00,0.142939
1,Albania,2001,8.00,628912.00,1.272038
2,Albania,2002,2.00,624323.00,0.320347
3,Albania,2003,4.00,611681.00,0.653936
4,Albania,2004,4.00,588886.00,0.679249
5,Albania,2005,0.00,567926.00,0.000000
6,Albania,2006,0.00,549994.00,0.000000
7,Albania,2007,4.00,529305.00,0.755708
8,Albania,2008,8.00,508752.00,1.572475
9,Albania,2009,0.00,489181.00,0.000000


,country,year,suicides,population,suicide100
0,Albania,2000,11.0,5.039000e+05,2.182973
1,Albania,2001,19.0,5.273980e+05,3.602592
2,Albania,2002,29.0,5.390370e+05,5.379965
3,Albania,2003,29.0,5.569440e+05,5.206987
4,Albania,2004,39.0,5.790360e+05,6.735333
5,Albania,2005,0.0,5.635970e+05,0.000000
6,Albania,2006,0.0,5.698170e+05,0.000000
7,Albania,2007,26.0,5.694290e+05,4.565977
8,Albania,2008,31.0,5.627750e+05,5.508418
9,Albania,2009,0.0,5.534510e+05,0.000000
